In [4]:
import os, sys
import scanpy as sc
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

In [2]:
from spamint import sprout_plus
from spamint import preprocess as pp
#import logging
#logger = logging.getLogger(__name__)
logger = sprout_plus.logger

# 1. Load files

In [5]:
logger.info("Loading files...")
inputDir = './scc_data/'
outDir = f'{inputDir}/results/'
sc_exp = pd.read_csv(f'{inputDir}/SC_exp.tsv',sep = '\t',header=0,index_col=0)
sc_meta = pd.read_csv(f'{inputDir}/SC_meta.tsv',sep = '\t',header=0,index_col=0)
st_exp = pd.read_csv(f'{inputDir}/ST_exp.tsv',sep = '\t',header=0,index_col=0)
st_coord = pd.read_csv(f'{inputDir}/ST_coord.csv',sep = ',',header=0,index_col=0)
st_decon = pd.read_csv(f'{inputDir}/ST_decon.tsv',sep = '\t',header=0,index_col=0)
sc_smurf = pd.read_csv(f'{inputDir}/smurf_ref.csv',sep = ',',header=0,index_col=0)
lr_df = pd.read_csv('./LR/human_LR_pairs.txt',sep='\t',header=None)
logger.info("File loading complete")

[2024-07-16 14:03:38,176/INFO] Loading files...


# Parameters

In [5]:
logger.info("Setting parameters...")
st_decon.columns = [x.split('sf_')[-1] for x in st_decon.columns]
species = 'Mouse'
st_tp = 'visum'
meta_key = 'level3_celltype'
SUM = 1e4
alpha, beta, gamma, delta, eta = [1, 0.001, 0.001, 0.1, 0.0005]
if st_tp == 'slide-seq':
    num_per_spot = 1
    repeat_penalty = 2
else:
    num_per_spot = 10
    repeat_penalty = int((st_exp.shape[0] * num_per_spot/sc_exp.shape[0]) * 10)

max_rep = 20

[2024-07-08 23:22:43,111/INFO] Setting parameters...


In [6]:
logger.debug("Reloading all SpaMint modules")
import importlib
l = [module for module in sys.modules.values() if module.__name__.startswith('spamint')]
for module in l:
    try:
        if module.__name__.startswith('spamint'):
            print(module.__name__)
            importlib.reload(module)
    except:
        pass

[2024-07-08 23:23:06,734/DEBUG] Reloading all SpaMint modules
spamint
spamint.utils
spamint.optimizers
spamint.preprocess
spamint.cell_selection
spamint.gradient_descent
spamint.sprout_plus


# Preprocess

In [7]:
if st_exp.shape[1]<1e4:
    # merfish data, only has 200~500 genes
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata_merfish(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                        sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)
else:
    sc_adata, st_adata, sc_ref, lr_df = pp.prep_all_adata(sc_exp = sc_exp, st_exp = st_exp, sc_distribution = sc_smurf, 
                                                            sc_meta = sc_meta, st_coord = st_coord, lr_df = lr_df, SP = species)

[2024-07-08 23:23:28,446/DEBUG] Data clean is done! Using 15399 shared genes .


In [8]:
obj_spex = sprout_plus.SpaMint(save_path = outDir, st_adata = st_adata, weight = st_decon, 
                sc_ref = sc_ref, sc_adata = sc_adata, cell_type_key = meta_key, lr_df = lr_df, 
                st_tp = st_tp)
#obj_spex.prep()

sc_ref and sc_adata has different genes, both data are subset to 15399 genes.
[2024-07-08 23:23:37,887/DEBUG] Parameters checked!
[2024-07-08 23:23:40,673/DEBUG] Getting svg genes
By setting k as 4, each spot has average 3.990990990990991 neighbors.
[2024-07-08 23:23:41,034/DEBUG] Calculating spots affinity profile data
[2024-07-08 23:23:43,459/DEBUG] SpaMint object created.


# cell selection

In [9]:
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0, mean_num_per_spot = num_per_spot,
                                    mode = 'strict', max_rep = 1,
                                    repeat_penalty = repeat_penalty)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

[2024-07-08 23:23:43,524/INFO] Starting cell selection
[2024-07-08 23:23:43,528/DEBUG] 0. calc num of cell per spot
[2024-07-08 23:23:43,530/DEBUG] 	 Estimating the cell number in each spot by the deconvolution result.
[2024-07-08 23:23:47,089/DEBUG] 1. filter gene
[2024-07-08 23:23:47,758/DEBUG] 2. feature select
[2024-07-08 23:27:40,629/DEBUG] 	 SpexMod selects 3747 feature genes.
[2024-07-08 23:27:40,633/DEBUG] 3. scale and norm
[2024-07-08 23:27:43,581/DEBUG] 4. init solution
[2024-07-08 23:29:11,621/DEBUG] 	 Init solution: max - 0.9201,     mean - 0.6428,     min - 0.1343
[2024-07-08 23:29:13,875/DEBUG] 5. Swap selection start...
[2024-07-08 23:29:13,876/DEBUG] 	Swap selection iter 0 of 1
[2024-07-08 23:33:01,940/DEBUG] 	 Swapped solution: max - 0.92,     mean - 0.71,     min - 0.20


if p != 0, extremely time consuming

In [ ]:
# change p to 0.1, use different code to select cells
sc_agg_meta = obj_spex.select_cells(use_sc_orig = True, p = 0.1, mean_num_per_spot = num_per_spot, mode = 'strict', max_rep = 1, 
                                    repeat_penalty = repeat_penalty)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)

In [10]:
alter_sc_exp, sc_agg_meta = obj_spex.gradient_descent(
                alpha, beta, gamma, delta, eta, 
                init_sc_embed = False,
                iteration = max_rep, k = 2, W_HVG = 2,
                left_range = 1, right_range = 2, steps = 1, dim = 2)
sc_agg_meta.to_csv(f'{outDir}/spexmod_sc_meta.tsv',sep = '\t',header=True,index=True)
with open(f'{outDir}/sc_knn.json', 'w') as fp:
    json.dump(obj_spex.sc_knn, fp)
# utils.save_object(obj_spex, f'{outDir}/obj_spex.pkl')


[2024-07-08 23:33:06,815/DEBUG] Running v12 now...
[2024-07-08 23:33:06,821/DEBUG] Init sc_coord by affinity embedding...
Average shape correlation is: 0.8824550524353375
[2024-07-08 23:34:37,614/DEBUG] First-term calculation done!
[2024-07-08 23:34:43,757/DEBUG] Second-term calculation done!


ValueError: No cell has neighbor, check parameter st_tp

In [30]:
from spamint import optimizers
importlib.reload(optimizers)
import cProfile
'''
print(optimizers.findCellKNN(obj_spex.st_coord, obj_spex.st_tp, 
                            obj_spex.sc_agg_meta,
                            obj_spex.gradient_descent_solver.sc_coord,
                            obj_spex.gradient_descent_solver.K ))
'''
self=obj_spex.gradient_descent_solver

logger.debug("calcNeighborAffMat")
#cProfile.runctx('''
optimizers.calcNeighborAffinityMat(self.spots_nn_lst, self.spot_cell_dict, self.lr_df, self.alter_sc_exp)
#                ''', globals(), locals(), sort='tottime')
logger.debug("OK")

logger.debug("calcAffMat")
optimizers.calculate_affinity_mat(self.lr_df, self.alter_sc_exp)
logger.debug("ok")

[2024-07-09 00:34:47,068/DEBUG] calcNeighborAffMat
[2024-07-09 00:35:04,701/DEBUG] OK
[2024-07-09 00:35:04,708/DEBUG] calcAffMat
[2024-07-09 00:35:18,771/DEBUG] ok
